# Phys 260: Python assignment header

### (1) Fill out the cell below.  
The cell below is a **code cell**.  Fill out your University of Michigan uniqname, then your name, and collaborators in the cell below **inside the quotes**.  

**Do not delete the quotes.**  We will use this information to organize your assignments.  To edit and execute cells, double click inside the cell, type, and press \<shift\>+\<enter\> to execute.

In [ ]:
UNIQNAME = ""
NAME = ""
COLLABORATORS = ""

### (2) Check your python version.  
**Execute the cell below** (double click in the cell and press \<shift\>+\<enter\>, or click in the cell and press the Run button) to check that you are using a version of python that is compatible with the tool we are using to grade your assignments.  If your ```IPython``` version is too old, we will *not* be able to grade your assignments.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

### (3) Do your best to answer all questions in the assignment.  
To answer questions, **replace** anything that says either
- "YOUR ANSWER HERE" 
- 
```
YOUR CODE HERE
raise NotImplementedError
``` 

with your answer/code.  Cells with either of the two bullet points above are cells of the notebook that will be graded.

**To edit markdown** cells (e.g. this one),  *double click in the cell to type*.  Press \<shift\>+\<enter\> to execute the cell.  


### (4) Make sure your notebook runs sequentially.
After you complete this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Introduction -- Reminder

Each Python lab will start with a pre-flight exercise that walks through building some of the set up and tools ($\sim$ 30 min), followed by an in-class tutorial with time for Q+A (50 min) so you can walk through steps that will be necessary for the homework assignment you will submit ($\sim$ 3 hrs).  Each lab will contain starter code, similar to what you see below.  Please fill in the code to complete the pre-flight assignment in preparation for the in-class tutorial.  

Preflight ($\sim$30-60 min, 10 points) **Typically due: Thursdays 3pm EST**

*Preflight typically graded by Wednesday 5p EST*

In-class tutorial and Q+A ($\sim$ 50 min, 10 points) **Typically occurs: Fridays 12pm EST**

Homework assignment ($\sim$ 3-5 hrs, 30 points) **Typically due: Tuesdays 12pm EST**  


When we grade your homework, we will not run your code. Once submitted, your notebook should have the outputs for all of your results.  Please do not include long outputs from debugging, beyond a few print statements and the requested visualimzations (i.e. plots).

**Grading:** When we grade your notebook, we will convert the .ipynb file to an HTML file.  We will be using [nbgrader](https://nbgrader.readthedocs.io/en/stable/) to grade your notebooks.  **Note:** Execute the cell below (click in the cell and press shift+enter, or click in the cell and press the Run button) to check that you are using a version of python that is compatible with the tool we are using to grade your assignments.  If your ```IPython``` version is too old, we will *not* be able to grade your assignments.

# Phys 260 Python Tutorial/HW 6: Simulating RC circuits (30 points total)

## Tutorial/HW summary
- **In-class tutorial**: Solving the SHO using the Euler Method
- **Homework**: Solve the differential equation of an RC circuit

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

## Equations of Motion:  Numerically solving differential equations

As we saw in the last Python homework before the exam, the real utility of using numerical methods in Python is solving complex problems which lack exact solutions or are very complicated to solve. This week we will learn how to solve differential equations, such as those that govern the time dependence of the electrical current in a circuit with resistors and capactiors. Although most simple circuits can be solved using a variety of techniques, more complex circuits, or more complex input signals, do not have easy paths to an exact solution.

### Solving first-order differential equations using Euler's method

Consider a first-order equation of the form:
\begin{equation}
\frac {dx} {dt} = f(x,t).
\end{equation}

The solution to this equation is a function $x(t)$ whose first derivative is $f$.
To find the solution to such an equation, we also need boundary conditions, or starting points.  Suppose that we have a first-order differential equation and know the value of $x$ for some specific time $t$, that is we know $x(t)$.  Then, we can write the value of x some short time later as
\begin{equation}
x(t+h) = x(t) + h \frac {dx} {dt} + \frac 1 2 h^2 \frac {d^2x} {dt^2} + ...
\end{equation}

\begin{equation}
x(t+h) = x(t) + h f(x,t) + h^2 \frac {df(x,t)} {dt} + ...
\end{equation}

If $h$ is small then $h^2$ is really small, and we can ignore it, and all higher terms of $h^n$.  This technique is called Euler's method.

Using this, if we know the value of $x$ at some time $t$ we can find the value at some short time later.  We can then repeat this process and understand how $x$ changes over time. The set of $x$ values at a discrete set of $t$ values is the numerical solution to our differential equation.

We are going to use this method to find the current, voltages and charges in simple circuits.  We will start with circuits which are easy to solve analytically so we can verify the numerical solution. Later we'll consider circuits that are a little more difficult.

### tl;dr

If we have a differential equation $\frac {dx} {dt} = f(x,t)$ and we know the starting value of $x(t)$, we can solve the equation numerically for a small time later $t+h$ using Euler's method, which is:

\begin{equation}
x(t+h) = x(t) + h f(x,t)
\end{equation}

By repeateadly performing this operation with small enough time steps, we can solve the differential equation for any time.

## Example from Mechanics: the Simple Harmonic Oscillator

### Reminder: Solution to undamped simple Harmonic oscillator

To illustrate these ideas, let's work out the solution to a situation with which you are already familiar: the simple harmonic oscillator, e.g. a mass on a spring:
<img src="http://www-personal.umich.edu/~gerdes/img/SHO.png" height="300" align="center">
This system is described by Newton's second law:
$$
    F = ma = m\frac{d^2x}{dt^2}= -kx.
$$

This is a second-order differential equation (it relates the function $x(t)$ to its second derivative), so we need to specify two boundary conditions. In mechanics, boundary conditions are often the initial position and velocity. 

Let's suppose that the mass starts from rest at $x_0=x(t=0)=1$. We know that the solution to Newton's second law is
\begin{eqnarray}
        x(t)& = & A\cos\omega t \\
        v(t) & = &-A\omega\sin\omega t \\
        a(t)& = &-A\omega^2\cos\omega t,
\end{eqnarray}
where $\omega = \sqrt{k/m}$, and in this case $A=1$.

### Setting up Euler's method for SHO with damping

Let's use Euler's method to find the solution when there is some viscous damping or friction that the mass experiences leading to a damping term, $cv$, that manifests in Newton's 2nd law as,
\begin{eqnarray}
    m\frac{d^2x}{dt^2}+c\frac{dx}{dt}+kx = 0\\
    ma + cv + kx = 0
\end{eqnarray}

Wait a minute, this is a second-order differential equation (it has both first *and* second derivatives of the position `x`)! We've only learned how to solve *first order* equations, so we need to find away to turn this into something of the form $\frac {dx} {dt} = f(x,t)$. We can reformulate the equation above as two coupled first-order differential equations. We can use the definitions of acceleration and velocity ($a = dv/dt$ and $v = dx/dt$). 

Rearanging these equations slightly, and putting them in discrete form:
$$
    \Delta v = a\Delta t,
$$
$$
    \Delta x = v\Delta t.
$$

Then, our equations for solving for v and x using Euler's method are:

$$
v(t+h) = v(t) + \Delta v = v(t) + a(t) h 
$$
$$
    x(t+h) = x(t) + \Delta x = x(t) + v(t) h
$$

In the first equation, $a(x,v,t)$ takes the place $f(x,t)$, and we know the equation for acceleration from above,
\begin{eqnarray}
    a = -cv/m - \omega x
\end{eqnarray}


Now that we have our discrete equations to solve the problem, let's get to coding.

### Plan of action


Our class will have two methods: 

`__init__` which will set the initial position, velocity, and time, and then calculate the initial acceleration, and also set useful constants like the time period and frequency of oscillations of the spring to attributes. **Below I ask you to fill in `self.omega` and `self.period`.**

In `__init__`, we will also initialize lists to keep track of the time, velocity, position, and acceleration as we evolve the system.

Second, `evolve_spring` puts Euler's method into action. `evolve_spring` take the number of periods we would like to simulate, as well as the number of time increments per period. We convert these to real time units of the incremental time (`self.dt`) and the total number of timesteps (`self.num_timesteps`).

We add the current time, velocity, position, and acceleration to their respective lists. Then, we update t_now, x_now, v_now, and a_now to their next values using the equations we derived above. **In `evolve_spring`, I've asked to fill in lines updating v_now and x_now, using the already updated acceleration.**


In [ ]:
class Spring : 
    """Creates an instance of a spring model, which can be time evolved"""
    def __init__(self, mass, spring_k, x0=1, v0=0, damping_c=0.1) :
        """A model of a spring
        Parameters
        ----------
        mass : float
            mass at end of spring
        spring_k : float 
            spring constant
        x0 : float
            initial position of mass. default 1
        v0 : float
            initial velocity of mass. default 0
        damping_c : float
            damping coefficient (c/m in our equations above) . default .1
     
        """
        self.x_now = x0
        self.v_now = v0
        self.a_now = -damping_c*v0 - spring_k * x0
        self.t_now = 0
        
        self.damping_c = damping_c
        self.spring_k = spring_k

        # Define the attributes omega (cycles per time) and the period (units of time) below
        # YOUR CODE HERE
        raise NotImplementedError()
        
        self.t_vals = []
        self.x_vals = []
        self.v_vals = []
        self.a_vals = []
        
               
    def evolve_spring(self, num_periods, num_timesteps_per_period) :
        """Evolve the spring, populate the acceleration, velocity and position (a_vals, v_vals, x_vals)
        Parameters
        ----------
        num_periods : float
            number of periods to evolve over
        num_timesteps_per_period : int
            number of timesteps per period, defines the time resolution         
        """
        
        
        ## setting the time increment (dt), and how many timesteps to calculate (num_timesteps)
        self.dt = self.period / num_timesteps_per_period
        self.num_timesteps = num_periods * num_timesteps_per_period
        
        for timestep in np.arange(self.num_timesteps) :
            # Populate
            self.a_vals.append(self.a_now)
            self.v_vals.append(self.v_now)
            self.x_vals.append(self.x_now)
            self.t_vals.append(self.t_now)
            
            # Keep tracking of the time
            self.t_now += self.dt

            # Update "now" values
            self.a_now = -self.damping_c*self.v_now - self.spring_k * self.x_now
            
            # Define attributes v_now based on a_now and dt, then x_now based on v_now and dt
            # YOUR CODE HERE
            raise NotImplementedError()



In [ ]:
"""Execute to check you're on the right track"""
test_spring = Spring(1, 1)
assert(test_spring.omega == 1)
assert(test_spring.period == 2*np.pi)

test_spring.evolve_spring(1, 100000)

assert(abs(test_spring.v_now - .0057) < 0.001)
assert(abs(test_spring.x_now - .73) < 0.01)

### Instantiating and using the `Spring` class (2 points)
Now, we create an instance of the `Spring` class to plot the behavior of a damped harmonic oscillator.  In the cell below, 
- Create an instance with name `spring`, `mass=1`, and `spring_k=1`. 
- Evolve the spring (using the appropriate method) for 10 periods, with 1000 timesteps per period.

In [ ]:
#  Create an instance of the spring and chage its state with time evolution
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert('dt' in dir(spring))
assert(abs(spring.dt - .006) <0.001 )

### Plot the behavior of the spring (2 points)

Plot $x$,$v$, and $a$ vs. time **(in units of the period)**


In [ ]:

fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(8,9))
fig.suptitle('Simple Harmonic Oscillator', fontsize=18)

# YOUR CODE HERE
raise NotImplementedError()
ax1.grid()
ax1.set_ylabel('position [m]', fontsize=14)

# YOUR CODE HERE
raise NotImplementedError()
ax2.grid()
ax2.set_ylabel('velocity [m/s]', fontsize=14)

# YOUR CODE HERE
raise NotImplementedError()
ax3.grid()
ax3.set_ylabel('acceleration [m/s$^2$]', fontsize=14)
ax3.set_xlabel('Time in units of period', fontsize=16)


### Limitations of Euler's method (class discussion)

In what scenarios might we expect Euler's method to fail? What are some ways you could figure out if your solution is reasonable?

YOUR ANSWER HERE

## RC circuit to model

Now we will use a very similar class and Euler's method to calculate the behavior of an electrical circuit.  Here, we will examine the behavior of:

- Case : An uncharged capacitor in series with a resistor and a battery that will be charged from the battery.  
<!-- - Case 2: A discharging capacitor as current flows through a resistor after the battery has been disconnected and the circuit closed.
 -->
<!-- In this case, the inital voltage across the capacitor is then zero, and the initial current through the resistor will be $I = \frac {V_b} {R}$ where $V_b$ is the voltage of the battery.  -->

The schematic for this circuit is below: 
<img src="http://www-personal.umich.edu/~gerdes/img/MC-PythonCircuit02.png" height="400" width="400">


First, we need to find our 'equation of motion'. The most obvious equation we can write down is using the loop rule.

\begin{eqnarray}
V_b - I(t)R - Q(t)/C = 0
\end{eqnarray}

where I is the current through the resistor and Q is the charge on the capacitor (technically +Q on the top plate and -Q on the bottom plate).


How can we make this look more like the SHO problem we solved previously? Because they are in series, the charge on the capacitor is directly related to the current through the resistor. 

$$
I(t) = \frac{dQ}{dt}
$$

To get our equation to look like something we can solve with Euler's method ( $\frac {dx} {dt} = f(x,t)$ ), let's plug this into the loop equation and solve for $\frac{dQ}{dt}$

$$
I(t) = \frac{dQ(t)}{dt} = \frac{1}{RC} [C V_b - Q(t)]
$$

<!-- We can write down the differential equation which includes this initial condition, and solve numerically.  Plot the current through the resistor and the charge on the capacitor as a function of time.  Following this, we can compute and plot the power delivered by the battery to the circuit as a function of time.  We can also compute the total energy delivered by the battery to the circuit, compare this to the energy stored in the capacitor energy dissipated as heat in the resistor? 
 -->




### Writing out the discrete versions of equations 

Here's a slightly different way of stating Euler's method for a generic differential equation of the form

$$
\frac {dx} {dt} = f(x,t)
$$

Let's say we know $x$ and $v=dx/dt$ at time $t_0$, which is in the $i$th step of our algorithm

$x_i = x(t_0)$ and $v_i = v(t_0)$

to get the values at $i+1$th step (or equivalently at time $t+ \Delta t$) we use Euler's method

$$
v_{i+1} = f(x_i,t_0+\Delta t)
$$

$$
x_{i+1} = x_i + v_i \Delta t 
$$


In preperation for filling out the RC circuit method, you may want to explicitly work these out for Q and I in the RC circuit case we discussed above.

## Build a class to model the RC circuit (4 points)

Below, I've given you the bones for the RCCircuit, which is very similar to Spring. 

Notice we've only specified the intial charge in the `__init__` method, because for a fixed battery voltage and charge on the capacitor, the initial current is already determined by our equation of motion. **First, you will set `self.i_now` in `__init__` , the initial current, given the initial charge and battery voltage.**

Also notice that instead of the period of oscillation which we used in the Spring class, we are using the RC constant as a characteristic time scale because the system does not oscillate.

The `evolve_circuit` method is also very similar to before, except now we are updating only two dependent variables: $Q and I$. In `evolve_circuit`, add the correct lines of code to update `self.q_now` and `self.i_now`.


In [ ]:
class RCCircuit :
    """Creates an instance of an rc circuit model, which can be time evolved"""
    def __init__(self, resistance, capacitance, q_capacitor, voltage_battery) :
        """A model of an rc circuit
        Parameters
        ----------
        resistance : float
            value of resistance in ohms
        capacitance : float 
            value of capacitance in Farads
        q_capacitor : float
            initial charge on capacitor
        voltage_battery : float
            voltage across battery
        """

        self.resistance = resistance
        self.capacitance = capacitance
        
        # The time constant is a characteristic timescale of the system, analogous to the period in a spring
        self.rc_constant = resistance * capacitance
        
        self.voltage_battery = voltage_battery
        self.q_now = q_capacitor
        self.t_now = 0
        
        #  Use the above equations to figure out the initial current through the resistor (self.i_now), 
        # using the initial charge and battery voltage.
        # YOUR CODE HERE
        raise NotImplementedError()
        
        self.i_vals = []
        self.q_vals = []
        self.t_vals = []
        
    def evolve_circuit(self, num_timescales, num_timesteps_per_timescale) :
        """Evolve the spring, populate the acceleration, velocity and position (a_vals, v_vals, x_vals)
        Parameters
        ----------
        num_periods : float
            number of periods to evolve over
        num_timesteps_per_period : int
            number of timesteps per period, defines the time resolution         
        """
        
        self.dt = self.rc_constant / num_timesteps_per_timescale
        self.num_timesteps = num_timescales * num_timesteps_per_timescale
        
        for timestep in np.arange(self.num_timesteps) :
            # Populate i_vals and q_vals with current current and charge on capacitor
            self.i_vals.append(self.i_now)
            self.q_vals.append(self.q_now)
            self.t_vals.append(self.t_now)
            
            # incrementing t_now
            self.t_now += self.dt
            
            # Using Euler's method and our equation of motion, update `self.q_now` and `self.i_now`. 
            # Take a look back at the Spring class if you are lost!
            # YOUR CODE HERE
            raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
test_rcc = RCCircuit(1,1e-6,0,10)
assert(test_rcc.i_now == 10)
test_rcc.evolve_circuit(10,100)
assert(test_rcc.i_now < 1e-3)

### Model the charging capacitor in an RC circuit (2 points)

In the cell below, 
- Define an instance of `RCCircuit` that corresponds to an RC circuit with the following initial conditions $R = 2\Omega$, $C = 20e-06$ Farad, $V_b=5$V, $Q_0=0$.  An appropriate name for this instance would be `rcc`.  
- Evolve the system over 5$\tau$ (i.e. $5 RC$), where $\tau$ is the relevant timescale with 1000 steps per $\tau$.

In [ ]:
# Define charging_rc_circuit here and evolve
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert('i_vals' in dir(rcc))
assert(rcc.dt == 4e-8)

### Calculate the power (2 points)
- Calculate the power output by the battery at each timestep, p_vals_battery
- Calculate the power dissipated by the resistor at each timestep, p_vals_resistor

Each of these depend on the attribute `i_vals`, which is a list of the current at each timestep.  You can either create an array from this list (recommended), or use a list comprehension if you want to practice this (and the power at each timestep would be a list).

In [ ]:
# Calculate the power here
# YOUR CODE HERE
raise NotImplementedError()
p_vals_battery.sum()
p_vals_resistor.sum()

In [ ]:
"""Execute to check you're on the right track"""
if type(p_vals_battery) == list :
    assert(np.array(p_vals_battery).sum() - 12415 < 1)
else :
    assert(p_vals_battery.sum() - 12415 < 1)

if type(p_vals_resistor) == list :
    assert(np.array(p_vals_resistor).sum() - 6252 < 1)
else :
    assert(p_vals_resistor.sum() - 6252 < 1)

### Plot the charge, current, and power delivered by the battery (2 points)

Below is most of the code to make a 3 panel plot of charge, current, and battery power as a function of time. Fill in a definition of the time normalized by the RC constant (`t_norm`) the relevant plot functions, and ylabels with units.


In [ ]:

fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(8,9))
fig.suptitle('Simple Harmonic Oscillator', fontsize=18)

# define t_norm as the time in units of the RC time constant

# YOUR CODE HERE
raise NotImplementedError()

# YOUR CODE HERE
raise NotImplementedError()
ax1.grid()


# YOUR CODE HERE
raise NotImplementedError()
ax2.grid()


# YOUR CODE HERE
raise NotImplementedError()
ax3.grid()

ax3.set_xlabel('Time in units of RC', fontsize=16)

**Discussion of figure** :  Let us talk through what is going on in the figure above.  Use the cell below to write a "figure caption" talking through the figure. (1 points)

YOUR ANSWER HERE

### Calculate the energy budget (4 points)

 We've already calculated the power delivered by the battery to the circuit as a function of time (converted from the chemical potential energy in the battery). Power is energy per unit time, so by integrating over the full time we can calculate the total energy deliver by/to different components.
 
Since we've already calculated the the battery power over time (`p_vals_battery`), we can calculate the total energy delivered to the circuit by the battery by numerically integrating, which I've done below by summing `p_vals_battery` and multiplying by the time increment.

Let's calculate the the energy stored in the capacitor, the energy lost to heat in the resistor, and the overall energy efficiency of our circuit.

Calculate:
- The energy stored on the capacitor, `energy_capacitor`, due to the collected charge by the end of the circuit's time evolution, 
- The energy dissipated by the resistor, `energy_resistor`, 
- The efficiency of the system (fraction of energy *not* lost to heat), `energy_efficiency`.

In [ ]:
# Calculate energy budget here

energy_battery = p_vals_battery.sum() * rcc.dt
# YOUR CODE HERE
raise NotImplementedError()
print("energy delivered by the battery: ", energy_battery)
print("energy stored in capacitor: ", energy_capacitor)
print("energy dissipated by resistor: ", energy_resistor)
print("fraction of energy not lost to heat: ", energy_efficiency)

In [ ]:
"""Execute to check you're on the right track"""
assert(energy_efficiency < 1)
assert(abs(1 - energy_capacitor/energy_resistor) < 0.05 )

**Comment on the energy budget** (2 points) : In the cell below, comment on the energy budget you calculated.  In particular, note how energy divides between the capacitor and the resistor.  Comment on why the energy efficiency is not 1.  Does the energy budget change for different values of R and C if we allow the system to evolve for a long time?  (Note, you can quickly test this to numerically check what happens.)

YOUR ANSWER HERE